In [60]:
import pandas as pd

data = pd.read_csv('../data/interim/training_data.csv')
data.head()

,reference,translation
0,shite i dont think theyre very happy,i dont think theyre very happy
1,if you want to understand animals i mean i mea...,if you want to understand game i mean really u...
2,sonia that smells awful,sonio that is a terrible thing
3,sam dolans father is out of his mind,sam dolans father checks out
4,theres a guy out there whos fucking serious,this is one very serious boy we have out there


In [62]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Define a dataset class
class TextDataset(Dataset):
    def __init__(self, toxic_texts, nontoxic_texts):
        self.toxic_texts = toxic_texts
        self.nontoxic_texts = nontoxic_texts
        self.vocab = self.build_vocab(self.toxic_texts + self.nontoxic_texts)
    
    def build_vocab(self, texts):
        # Tokenize text and create a vocabulary
        counter = Counter()
        for text in texts:
            counter.update(word_tokenize(text.lower()))
        # Create a vocab dictionary that maps words to unique indices, start from 1 for padding
        vocab = {word: i+1 for i, (word, _) in enumerate(counter.most_common())}
        # Add a special token for unknown words
        vocab['<UNK>'] = len(vocab) + 1
        return vocab
    
    def text_to_sequence(self, text):
        # Convert text to a sequence of integers
        tokens = word_tokenize(text.lower())
        return [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]

    def __len__(self):
        return len(self.toxic_texts)
    
    def __getitem__(self, idx):
        toxic_seq = self.text_to_sequence(self.toxic_texts[idx])
        nontoxic_seq = self.text_to_sequence(self.nontoxic_texts[idx])
        return torch.tensor(toxic_seq), torch.tensor(nontoxic_seq)

    @staticmethod
    def collate_fn(batch):
        toxic_seqs, nontoxic_seqs = zip(*batch)
        toxic_seqs_padded = pad_sequence(toxic_seqs, batch_first=True, padding_value=0)
        nontoxic_seqs_padded = pad_sequence(nontoxic_seqs, batch_first=True, padding_value=0)
        return toxic_seqs_padded, nontoxic_seqs_padded

# Instantiate the dataset
text_dataset = TextDataset(data['reference'].tolist(), data['translation'].tolist())


[nltk_data] Downloading package punkt to /Users/random/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
data_loader = DataLoader(text_dataset, batch_size=4, shuffle=True, collate_fn=TextDataset.collate_fn)

# Let's check the vocabulary size
vocab_size = len(text_dataset.vocab)
vocab_size

(tensor([  23,    2,   49,    4,  453,  807,    6,  175,    6,  175,    2,  203,
           21, 3232]),
 tensor([  23,    2,   49,    4,  453,  536,    6,  175,  125,  453,   57,    2,
          196,   21,    3, 2731]))

In [64]:
import torch.nn as nn

class DetoxifyModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(DetoxifyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        x = self.embedding(x)
        output, (hidden, cell) = self.rnn(x)
        x = self.fc(output)
        return x

In [ ]:
model = DetoxifyModel(vocab_size=vocab_size, embedding_dim=)

In [65]:
def train(model, data_loader, loss_fn, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        for toxic_seqs, nontoxic_seqs in data_loader:
            optimizer.zero_grad()
            output = model(toxic_seqs)
            loss = loss_fn(output.transpose(1, 2), nontoxic_seqs)
            loss.backward()
            optimizer.step()
            print(f'Epoch {epoch}, Loss: {loss.item()}')